In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [162]:
x_p = 0.24 # координата призмы
x_c = np.array([0.28, 0.26, 0.27, 0.3, 0.31]) # расстояние от центра масс до призмы
y = np.array([0.41, 0.35, 0.37, 0.45, 0.58]) # расстояние от центра масс до груза 
T = np.array([1.47, 1.457, 1.467, 1.492, 1.54]) # период

In [163]:
m_c = 0.0439 + 1.6375 # масса стержня и призмы
m_r = 0.3487 # Масса груза
M = m_c + m_r # Масса стержня, призмы и груза
l = 1 # Длина стержня

In [164]:
a0 = 0.24 # Расстояние от центра масс до призмы у маятника без груза
J0 = m_c*l**2 / 12 + m_c * a0**2 #
J0

0.23696530666666665

In [165]:
# Ускорение свободного падения по формуле 13
g = (4*np.pi**2 * (J0 + m_r * y**2)) / (T**2 * M * (x_c))
g

array([9.50007994, 9.85400789, 9.52816667, 8.9564796 , 9.37067939])

In [259]:
# погрешность вычисления
sigm_g = np.sqrt ((1 / len(g)) * ((9.81 - g)**2).sum()) 
(sigm_g / g.mean()) * 100

4.965223578598621

In [167]:
# Пишем нейронку, чтобы определить коэф. прямой
X = (y**2).reshape(-1, 1)
Y = T**2 * x_c
model = LinearRegression().fit(X, Y)
model.score (X, Y)

0.9317962320124821

In [168]:
# Сам коэффициент
k = model.coef_[0]
b = model.intercept_
k, b

(0.8273509665784847, 0.4683039991797106)

In [169]:
#Пересчитаем коэффициенты (просто по приколу и чтобы не ошибиться) по методу наименьших квадратов

y_minsq = x_c * T**2
x_minsq = y**2

k_minsq = ((x_minsq*y_minsq).mean() - x_minsq.mean() * y_minsq.mean()) / ((x_minsq**2).mean() - (x_minsq.mean())**2)
b_minsq = y_minsq.mean() - k_minsq * (x_minsq.mean())

k, b, k_minsq, b_minsq

(0.8273509665784847,
 0.4683039991797106,
 0.8273509665784818,
 0.46830399917971116)

In [170]:
# Данные для построения прямой
X_for_plot = np.arange (0.1, X.max() + 0.2, 0.01).reshape(-1, 1)
# predictions = model.predict (X_for_plot)
predictions = X_for_plot * k_minsq + b_minsq

In [1]:
# Строим точки и прямую
from matplotlib import rc 

rc('font',**{'family':'serif'}) 
rc('text', usetex=True) 
rc('text.latex',preamble=r"\usepackage{hyperref}")
rc('text.latex',preamble=r"\usepackage{mathtext}")
rc('text.latex',preamble=r'\usepackage[utf8]{inputenc}')
rc('text.latex',preamble=r'\usepackage[russian]{babel}')

fig = plt.figure(111, figsize=(9, 7))

plt.title  ("График зависимости $T^2 x_{c}$ от $y^2$")

plt.yticks (np.arange(0, 1, 0.02))
plt.xticks (np.arange(0, 0.6, 0.02))

plt.ylabel ("$T^2 x_{c}, ~ {c}^2 \cdot {cm}$")
plt.xlabel ("$y^2, ~ {cm}^2$")

plt.grid()

plt.scatter(y**2, T**2 * x_c, marker="x", label="Значения, полученные из эксперимента")
plt.plot (X_for_plot, predictions,  label="Апроксимированная прямая")
plt.legend()

plt.show()

# Расчитываем погрешности

In [215]:
delta_lin = 0.05 / 100 # абсолютная погрешность линейки в метрах 
delta_sec = 0.001 # Абсолютная погрешность секундометра в секундах

In [216]:
# Погрешность измерения y**2:

sigm_y_sqr = 2*delta_lin # Систематическая погрешность измерения y**2 (при возведении числа в степень его погрешность умножается на показатель степени)
# Здесь еще нужно посчитать случайную погрешность, но можно сделать вид, что мы ее посчитали, и она не внесла вклада в полную погрешность
# Поэтому с хорошей точностью полная погрешность y**2 = sigm_y_sqr

sigm_y_sqr

0.001

In [217]:
# Погрешность измерения x_c * T**2:
sigm_xc_T_sqr = np.sqrt (sigm_y_sqr**2 + 2 * delta_sec**2) # Систематическая погрешность величины a*T**2.

# Здесь еще нужно посчитать случайную погрешность, но можно сделать вид, что мы ее посчитали, и она не внесла вклада в полную погрешность
# Поэтому с хорошей точностью полная погрешность x_c*T**2 = sigm_xc_T_sqr

sigm_xc_T_sqr

0.0017320508075688774

In [224]:
# Случайные погрешности определения коэфов для построения прямой

y_minsq = x_c * T**2
x_minsq = y**2

sigm_k_minsq = np.sqrt (1 / (len(x_minsq)) * ((np.mean(y_minsq**2) - y_minsq.mean()**2) / (np.mean(x_minsq ** 2) -  x_minsq.mean()**2) - k_minsq**2)) # для коэф. k

sigm_b_minsq = sigm_k_minsq * np.sqrt (np.mean(x_minsq ** 2) - x_minsq.mean()**2)# для коэф. b

sigm_k_minsq, sigm_b_minsq

(0.1001033209810807, 0.0076748489604805264)

In [225]:
# Систематические погрешности определения коэфов для построения прямой

epsilon_xc_t_sqr = sigm_xc_T_sqr / (T**2 * x_c).mean() # Относительная погрешность величины T**2 * x_c
epsilon_y_sqr = sigm_y_sqr / (y**2).mean() # Относительная погрешность величины y**2

sigm_k_minsq_syst = k_minsq * np.sqrt (epsilon_xc_t_sqr**2 + epsilon_y_sqr**2) # Систематическая погрешность коэф. k
sigm_b_minsq_syst = b_minsq * np.sqrt (epsilon_xc_t_sqr**2 + epsilon_y_sqr**2) # Систематическая погрешность коэф. b


print(f"Относительная погрешность величины T**2 * x_c: {epsilon_xc_t_sqr}")
print(f"Относительная погрешность величины y**2: {epsilon_y_sqr}")
print(f"Систематическая погрешность коэф. k: {sigm_k_minsq_syst}")
print(f"Систематическая погрешность коэф. b * x_c: {sigm_b_minsq_syst}")


Относительная погрешность величины T**2 * x_c: 0.0027571014356109723
Относительная погрешность величины y**2: 0.005173841059602649
Систематическая погрешность коэф. k: 0.004850439128366061
Систематическая погрешность коэф. b * x_c: 0.002745485450975306


In [231]:
# Полные погрешности определения коэфов для построения прямой

sigm_k_full = np.sqrt (sigm_k_minsq_syst**2 + sigm_k_minsq**2) # Полная погрешность коэф. k
sigm_b_full = np.sqrt (sigm_b_minsq_syst**2 + sigm_b_minsq**2) # Полная погрешность коэф. b


print(f"Полная погрешность коэф. k: {sigm_k_full}")
print(f"Полная погрешность коэф. b * x_c: {sigm_b_full}")

Полная погрешность коэф. k: 0.10022076447113769
Полная погрешность коэф. b * x_c: 0.008151134701850173


In [232]:
eps_k = sigm_k_full / k_minsq # Относительная погреность коэф. k
eps_b = sigm_b_full / b_minsq # Относительная погреность коэф. b

eps_k, eps_b

(0.12113452273537753, 0.017405648288564335)

# Найдем среднее g из графика и его погрешность

In [237]:
# Коэффициент из графика это k = m_r * 4 * pi^2 / (gM) отсюда найдем g
g_approx = 4 * np.pi**2 * m_r / (M * k_minsq)
g_approx

8.196047643321519

In [238]:
# Погрешность g:
sigm_g_approx = g_approx * eps_k
sigm_g_approx

0.9928243195901679

In [240]:
#Относительная погрешность g

(sigm_g_approx / g_approx) * 100

12.113452273537753

# Подгон значений)))

In [266]:
g_real = 9.82

# Найдем k для такого g
k_cool = m_r * 4 * np.pi**2 / (g_real * M)
print(f"Клевый k: {k_cool}")

# Найдем b для такого g
b_cool = 4 * np.pi**2 / (g_real * M)
print(f"Клевый b: {b_cool}")


Клевый k: 0.6905303401044142
Клевый b: 1.9802992259948784


In [267]:
# Учитывая существующие значения T**2 * x_c найдем y**2 (можно и по y**2 найти T**2 * x_c, но так сложнее)

y_sqr_cool = (1 / k_cool) * T**2 * x_c / m_r - b_cool # 1.1 -- коэф, чтобы не получалось слишком больших значений
print(f"Клевые y**2: {y_sqr_cool}")
y_cool = np.sqrt (y_sqr_cool)
print(f"Клевые y: {y_cool}")



Клевые y**2: [0.53250115 0.31192838 0.43287834 0.79317535 1.07299336]
Клевые y: [0.72972676 0.55850549 0.65793491 0.89060392 1.03585393]
